# `writer.py`
**Writer Process file**

In [ ]:

import json
from pathlib import Path

from rich.console import Console
from rich.prompt import Prompt
console = Console()

SHARED_STATE_FILE = Path(__file__).parent / "ipc_state.json"


#======================================================================
# Shared State Read/Write Functions
def read_shared_state():
    """
    Read the shared state from the JSON file.
    return: dict - The current state dictionary
    """
    try:
        with open(SHARED_STATE_FILE, "r") as file:
            return json.load(file)
    except FileNotFoundError:
        print("Shared state file not found.")

def write_shared_state(state):
    """
    Write the shared state to the JSON file.
    param state: dict - The state dictionary to write
    """
    with open(SHARED_STATE_FILE, "w") as file:
        json.dump(state, file, indent=2)
#======================================================================

### Menu Handling Functions

#### Add New Account

#### Add New Transaction

#### View Transactions for an Account

In [ ]:
# Option 5: View Transactions for an Account
def view_account_transactions(console):
    state = read_shared_state()
    accounts = state.get("accounts", [])
    account = choose_account(accounts, console)
    transactions = account.get("transactions", [])
    console.print(f"\n[yellow]Transactions for account '{account['name']}':[/yellow]")
    if transactions:
        for i, transaction in enumerate(transactions, start=1):
            console.print(f"{i}. Amount: {transaction['amount']}, Description: {transaction['description']}")
    else:
        console.print("[red]No transactions available for this account.[/red]")


#=================================================================================
# From writer_layout.py
def choose_account(accounts, console):
    """
    Choose an account from a list of accounts
    param account: dict - The account dictionary
    param console: Console - The Rich Console object to use for output
    """
    account_names = [account['name'] for account in accounts]
    account_name = Prompt.ask("Select Account", choices=account_names)

    #=======================================================================================
    # Generator Expression:
        # Iterates through each account to find the matching account name
        # filtered to only include accounts where the name key matches account_name
        # Yields matching account dictionaries
        # next(...) is a iterator function that retrieves the first item from the generator
            # Stops immediately after finding the first match (efficient!)
            # None - This is the default value:
                # If no matching account is found, next() returns None instead of raising a StopIteration exception
    #=======================================================================================
    # Finds and returns the first account dictionary where account['name'] equals account_name. If no match is found, it returns None.
    account = next((acc for acc in accounts if acc['name'] == account_name), None)


    console.print(f"\n[bold blue]Selected Account: {account_name}[/bold blue]")
    return account

#### Change View

In [ ]:
# Option 3 and 4: View Summary/ View Accounts
def change_current_view(new_view):
    """
    Change the current view in the shared state.
    param new_view: str - The new view to set
    """
    state = read_shared_state()
    if state:
        state["current_view"] = new_view
        write_shared_state(state)

#### Menu Handler
**Uses**
- `new_account(console)`
- `new_transaction(console)`
- `change_current_view("SUMMARY")`
- `view_account_transactions(console)`

In [ ]:
def handle_menu__choice(choice):
    """
    Handle the user's menu choice
    param choice: str - The user's menu choice
    return: int - 0 to exit, else None
    """
    match choice:
        case "0":
            console.print("\n[yellow]Exiting Command Client. Goodbye! 👋[/yellow]\n")
            return 0
        case "1":# Add New Account
            new_account(console)
        case "2":# Add New Transaction
            new_transaction(console)
        case "3":# View Summary
            console.print("\n[yellow]Viewing Summary! 👋[/yellow]\n")
            change_current_view("SUMMARY")
        case "4":# View Accounts
            console.print("\n[yellow]Viewing Accounts! 👋[/yellow]\n")
            change_current_view("ACCOUNTS")
        case "5":# View Account Transactions
            view_account_transactions(console)
        case _:
            console.print("[red]❌Invalid choice. Please try again.[/red]")